In [1]:
import pyodbc
import sqlite3
import os.path
import sys
sys.path.append(r'/home/ernst/Projects/Prometheon/rest-api-azure')
# from numpy import product as npproduct
# from numpy import array as nparray
from datetime import datetime
from numpy import product as npproduct
from numpy import array as nparray
from project_config.config import (sql_parameter_tablename_dict,
                                  required_keys,
                                  base_rate,
                                  db_path,
                                  AZURE_SERVER,
                                  AZURE_USERNAME,
                                  AZURE_PASSWORD,
                                  AZURE_DATABASE,
                                  AZURE_PORT)

In [2]:
def query_azure(sql, values):
    driver= '{ODBC Driver 13 for SQL Server}'
    cnxn = pyodbc.connect((r"DRIVER={driver};PORT={port};SERVER={server};"
                           r"DATABASE={database};UID={username};PWD={password}").format(driver=driver,
                                                    port=AZURE_PORT,
                                                    server=AZURE_SERVER,
                                                    database=AZURE_DATABASE,
                                                    username=AZURE_USERNAME,
                                                    password=AZURE_PASSWORD))
    cursor = cnxn.cursor()
    try:
        # cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
        cursor.execute(sql, values)
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        return(columns, rows)
    except Exception as e:
        print(e)
    finally: # Always executed, even in case of exception
        cnxn.close()
        

In [165]:
sql1 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID > 1200 and  Reference_Data_ID < 1300 and Product_ID = 1'
sql2 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID = 577 and Product_ID = 2'
sql3 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID = ? and Product_ID = ?'

In [166]:
sql_unique = 'SELECT COUNT(1), Reference_Data_Type FROM [dbo].[Pricing_Param_Ref_Data] GROUP BY Reference_Data_Type'


In [167]:
columns, row = query_azure(sql_unique, [])

In [170]:
columns, row = query_azure(sql1, [])

In [171]:
columns

['Reference_Data_ID',
 'Product_ID',
 'Pricing_Parameter_Value',
 'Product_Description',
 'Pricing_Parameter_Type_ID',
 'Pricing_Parameter_Type_Name',
 'Pricing_Parameter_ID',
 'Pricing_Parameter_Name',
 'Reference_Data_Type',
 'Reference_Data_Name']

In [172]:
row

[(1201, 1, 1.5, 'Temperature only', 5, 'Country', 14, 'Medium risk', 'Country', 'Chile'),
 (1202, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'China'),
 (1203, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'Hong Kong, SAR China'),
 (1204, 1, 2.0, 'Temperature only', 5, 'Country', 23, 'High risk', 'Country', 'Macao, SAR China'),
 (1205, 1, 2.0, 'Temperature only', 5, 'Country', 23, 'High risk', 'Country', 'Christmas Island'),
 (1206, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'Cocos (Keeling) Islands'),
 (1207, 1, 1.5, 'Temperature only', 5, 'Country', 14, 'Medium risk', 'Country', 'Colombia'),
 (1208, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'Comoros'),
 (1209, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'Congo (Brazzaville)'),
 (1210, 1, 1.0, 'Temperature only', 5, 'Country', 5, 'Low risk', 'Country', 'Congo, (Kinshasa)'),
 (1211, 1, 1.0, 'Temperature only', 5, 'Co

In [158]:
from project_config.config import (sql_parameter_tablename_dict,
                                  required_keys,
                                  base_rate,
                                  db_path,
                                  AZURE_SERVER,
                                  AZURE_USERNAME,
                                  AZURE_PASSWORD,
                                  AZURE_DATABASE,
                                  AZURE_PORT)
valid_pricing_parameter_type_names = ['country', 'airport', 'container type']

In [159]:
sql_parameter_tablename_dict.keys()

dict_keys(['airports', 'ground_handlers'])

In [160]:
driver= '{ODBC Driver 13 for SQL Server}'
def calculate_premium(premium_request):
    """
    :premium_request : a dictionary

    Looks into the database to search for the parameters, and appends the
    results to a DataFrame.
    Returns the quote amount, and a list of dictionaries with detailed info
    """
    ## Assert that all required keys to calculate a premium are provided
    for key in required_keys:
        assert key in premium_request.keys(), ('Aborting: key "{}" missing in '
                                            'request'.format(key))

    ## Get values from request: timestamp, amount insured, product_id
    # timestamp 
    try:
        # The following works when a list is given of [Y, M, D]
        timestamp = datetime(*premium_request['timestamp'])
    except:
        # When input is a string: 'yyyy-mm-dd'
        timestamp = datetime.strptime(premium_request['timestamp'], '%Y-%m-%d')
    
    # To filter valid_from/to (not implemented yet)
    timestamp_str = datetime.strftime(timestamp, '%Y-%m-%d')
        
    # product_id and amount 
    product_id = premium_request['product_id']
    sum_insured = premium_request['sum_insured']  
    
    # init the list that contains the elements that determine the premium
    quote_reply = [{'sum_insured': sum_insured,
                   'product_id': product_id,
                   'timestamp': timestamp_str}]

    # Loop through all items, to search for the "parameters"/premium multipliers
    for key, parameter_ids in premium_request.items():
        
        # Test if this is a key that requires a parameter lookup
        # if not, skip it
        if key in valid_pricing_parameter_type_names:
            sql = """ SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  """
            
            # Make database connection
            db_connection = pyodbc.connect((r"DRIVER={driver};PORT={port};SERVER={server};"
                           r"DATABASE={database};UID={username};PWD={password}").format(driver=driver,
                                                    port=AZURE_PORT,
                                                    server=AZURE_SERVER,
                                                    database=AZURE_DATABASE,
                                                    username=AZURE_USERNAME,
                                                    password=AZURE_PASSWORD))
            # Create database cursor
            cursor = db_connection.cursor()
            try:
                for parameter_id in parameter_ids:
                    vals = (parameter_id, product_id) #Timestamp not used yet!
                    cursor.execute(sql, vals)
                    
                    # fetch the result
                    columns = [column[0] for column in cursor.description]
                    
                    # find the index of the row where the Pricing Parameter Value is located
                    # NB: those strings should probably be in config
                    pricing_param_value_index = [i for i, col_name in enumerate(columns) 
                                       if col_name == 'Pricing_Parameter_Value'][0]
                    ref_data_name_index = [i for i, col_name in enumerate(columns) 
                                       if col_name == 'Reference_Data_Name'][0]
                    pricing_param_type_name_index = [i for i, col_name in enumerate(columns) 
                                       if col_name == 'Pricing_Parameter_Type_Name'][0] 
                    
                    # Get the values
                    row = cursor.fetchone()
                    pricing_param_value = row[pricing_param_value_index]  
                    ref_data_name = row[ref_data_name_index]  
                    pricing_param_type_name = row[pricing_param_type_name_index] 
                    
                    # Append dictionaries to the list


                    quote_reply.append({'type': key,
                                        'pricing_param_type_name': pricing_param_type_name,
                                        'pricing_param_value': pricing_param_value,
                                        'ref_data_name': ref_data_name})
            except Exception as e:
                print(e)
                return {'message': ('parameter_id {} for key {} for date'
                                '{} not found').format(
                                    parameter_id, key, timestamp_str)}
            finally:
                db_connection.close()
        else:
            continue # for clarity

    # Process the obtained data to calculate the premium
    parameters = [item['pricing_param_value'] for item in quote_reply if 'pricing_param_value' in item.keys() ]
    # Multiply all parameters
    parameters_product = 1
    for par in parameters:
        parameters_product *= par
    premium = (premium_request['sum_insured'] * base_rate * parameters_product)
    return (premium, quote_reply)

In [173]:
premium_request = { 'product_id': 1,
                    'sum_insured' : 360E3,
                    'container type' : [1147],
                    'airport' : [651, 652, 890],
                    'timestamp' : (2018, 1, 7),
                      'country': [1217]} #Y, M, D
premium, quote_request = calculate_premium(premium_request)
    


premium request: {'product_id': 1, 'sum_insured': 360000.0, 'container type': [1147], 'airport': [651, 652, 890], 'timestamp': (2018, 1, 7), 'country': [1217]}
------------------------
key: container type, parameter ids: [1147]
parameter id: 1147
sql:  SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  
vals: (1147, 1)
------------------------
key: airport, parameter ids: [651, 652, 890]
parameter id: 651
sql:  SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  
vals: (651, 1)
parameter id: 652
sql:  SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  
vals: (652, 1)
parameter id: 890
sql:  SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  
vals: (890, 1)
----------------

In [186]:
def tf():
    return 1

In [187]:
a = tf()

In [188]:
rr, rrr = a

TypeError: 'int' object is not iterable

In [185]:
rrr

2

In [38]:
for key, parameter_ids in premium_request.items():
    print(key, parameter_ids)

product_id 1
sum_insured 360000.0
ground_handlers [1, 5, 10]
airports [650, 652, 890]
timestamp (2018, 1, 7)


In [ ]:
p_0 =1
for par in parameters:
    p_0 *= par

In [ ]:
p_0


In [ ]:
print(premium)
print(quote_reply)

(1351, 2, 1.8, 'Freight insurance', 5, 'Country', 14, 'Medium risk', 'Country', 'Saudi Arabia'),

(1351, 1, 1.5, 'Temperature only', 5, 'Country', 14, 'Medium risk', 'Country', 'Saudi Arabia')
